# Start-to-Finish Example: Head-On Black Hole Collision

### Author: Patrick Nelson

### Adapted from [Start-to-Finish Example: Head-On Black Hole Collision](Tutorial-Start_to_Finish-BSSNCurvilinear-Two_BHs_Collide.ipynb)

## This module implements a basic GRFFE code to evolve a toy model of a neutron star magnetosphere.

### NRPy+ Source Code for this module: 
1. [GiRaFFEfood_HO/GiRaFFEfood_HO_AlignedRotator.py](../edit/GiRaFFEfood_HO/GiRaFFEfood_HO_AlignedRotator.py); [\[**tutorial**\]](Tutorial-GiRaFFEfood_HO_Aligned_Rotator.ipynb): Aligned rotator initial data, sets all FFE variables in a Cartesian basis.
1. [GiRaFFE_HO/GiRaFFE_HO_.py](../edit/GiRaFFEfood_HO/GiRaFFEfood_HO_AlignedRotator.py); [\[**tutorial**\]](Tutorial-GiRaFFEfood_HO_Aligned_Rotator.ipynb): Generates the right-hand sides for the GRFFE evolution equations in Cartesian coordinates.
We will also borrow C code from the ETK implementation of $\text{GiRaFFE_HO}$

Here we use NRPy+ to generate the C source code necessary to set up initial data for a model neutron star (see [the original GiRaFFE paper](https://arxiv.org/pdf/1704.00599.pdf)). Then we use it to generate the RHS expressions for [Method of Lines](https://reference.wolfram.com/language/tutorial/NDSolveMethodOfLines.html) time integration based on the [explicit Runge-Kutta fourth-order scheme](https://en.wikipedia.org/wiki/Runge%E2%80%93Kutta_methods) (RK4).

The entire algorithm is outlined below, with NRPy+-based components highlighted in <font color='green'>green</font>.

1. Allocate memory for gridfunctions, including temporary storage for the RK4 time integration.
1. (**Step 2** below) <font color='green'>Set gridfunction values to initial data (**[documented in previous start-to-finish module](Tutorial-Start_to_Finish-BSSNCurvilinear-Setting_up_two_BH_initial_data.ipynb)**).</font>
1. Evolve the initial data forward in time using RK4 time integration. At each RK4 substep, do the following:
    1. (**Step 3A** below) <font color='green'>Evaluate BSSN RHS expressions.</font>
    1. (**Step 4** below) Apply singular, curvilinear coordinate boundary conditions [*a la* the SENR/NRPy+ paper](https://arxiv.org/abs/1712.07658)
1. (**Step 3B** below) At the end of each iteration in time, output the <font color='green'>FFE variables</font>. (This is in Step 3B, because Step 4 requires that *all* gridfunctions be defined.)
1. Repeat above steps at two numerical resolutions to confirm convergence to the expected value.

In [1]:
# First we import needed core NRPy+ modules
from outputC import *
import NRPy_param_funcs as par
import grid as gri
import loop as lp
import indexedexp as ixp
import finite_difference as fin
import reference_metric as rfm

# Set spatial dimension (must be 3 for BSSN)
DIM = 3
par.set_parval_from_str("grid::DIM",DIM)

# Then we set the coordinate system for the numerical grid
par.set_parval_from_str("reference_metric::CoordSystem","Cartesian")
rfm.reference_metric() # Create ReU, ReDD needed for rescaling B-L initial data, generating BSSN RHSs, etc.

# Then we set the phi axis to be the symmetry axis; i.e., axis "2", corresponding to the i2 direction. 
#      This sets all spatial derivatives in the phi direction to zero.
#par.set_parval_from_str("indexedexp::symmetry_axes","2") # Let's not deal with this yet.

#################
# Next output C headers related to the numerical grids we just set up:
#################

# Create directories for the thorn if they don't exist.
!mkdir GiRaFFE_standalone 2>/dev/null # 2>/dev/null: Don't throw an error if the directory already exists.

# First output the coordinate bounds xxmin[] and xxmax[]:
with open("GiRaFFE_standalone/xxminmax.h", "w") as file:
    file.write("const REAL xxmin[3] = {"+str(rfm.xxmin[0])+","+str(rfm.xxmin[1])+","+str(rfm.xxmin[2])+"};\n")
    file.write("const REAL xxmax[3] = {"+str(rfm.xxmax[0])+","+str(rfm.xxmax[1])+","+str(rfm.xxmax[2])+"};\n")

# Next output the proper distance between gridpoints in given coordinate system.
#     This is used to find the minimum timestep.
dxx     = ixp.declarerank1("dxx",DIM=3)
ds_dirn = rfm.ds_dirn(dxx)
outputC([ds_dirn[0],ds_dirn[1],ds_dirn[2]],["ds_dirn0","ds_dirn1","ds_dirn2"],"BSSN/ds_dirn.h")

# Generic coordinate NRPy+ file output, Part 2: output the conversion from (x0,x1,x2) to Cartesian (x,y,z)
outputC([rfm.xxCart[0],rfm.xxCart[1],rfm.xxCart[2]],["xCart[0]","xCart[1]","xCart[2]"],
        "BSSN/xxCart.h")

Wrote to file "BSSN/ds_dirn.h"
Wrote to file "BSSN/xxCart.h"


## Step 2A: Import Aligned Rotator initial data C function

The [GiRaFFEfood_HO.GiRaFFEfood_HO_AlignedRotator.py](../edit/GiRaFFEfood_HO/GiRaFFEfood_HO_AlignedRotator.py) NRPy+ module does the following:

1. Set up Aligned rotator initial data quantities in the **Cartesian basis**, as [documented here](Tutorial-GiRaFFEfood_HO_Aligned_Rotator.ipynb). 


In [2]:
import GiRaFFEfood_HO.GiRaFFEfood_HO_Aligned_Rotator as gfAR
gfAR.GiRaFFEfood_HO_Aligned_Rotator()

# Step 2: Create the C code output kernel.
# To best format this for the ETK, we'll need to register this gridfunction.
ValenciavU = ixp.register_gridfunctions_for_single_rank1("AUX","ValenciavU")
#BU = ixp.register_gridfunctions_for_single_rank1("AUX","BU")
GiRaFFEfood_A_v_to_print = [\
                            lhrh(lhs=gri.gfaccess("out_gfs","AD0"),rhs=gfAR.AD[0]),\
                            lhrh(lhs=gri.gfaccess("out_gfs","AD1"),rhs=gfAR.AD[1]),\
                            lhrh(lhs=gri.gfaccess("out_gfs","AD2"),rhs=gfAR.AD[2]),\
                            lhrh(lhs=gri.gfaccess("out_gfs","ValenciavU0"),rhs=gfAR.ValenciavU[0]),\
                            lhrh(lhs=gri.gfaccess("out_gfs","ValenciavU1"),rhs=gfAR.ValenciavU[1]),\
                            lhrh(lhs=gri.gfaccess("out_gfs","ValenciavU2"),rhs=gfAR.ValenciavU[2]),\
                            ]

GiRaFFEfood_A_v_CKernel = fin.FD_outputC("returnstring",GiRaFFEfood_A_v_to_print,params="outCverbose=False")

# Format the code within a C loop over cctkGH
#GiRaFFEfood_A_v_looped = loop.loop(["i2","i1","i0"],["0","0","0"],["cctk_lsh[2]","cctk_lsh[1]","cctk_lsh[0]"],\
#                                   ["1","1","1"],["#pragma omp parallel for","",""],"",\
#                                   GiRaFFEfood_A_v_CKernel.replace("time","cctk_time"))

# Step 4: Write the C code kernel to file.
with open("GiRaFFE_standalone/GiRaFFEfood_A_v_AlignedRotator.h", "w") as file:
    file.write(str(GiRaFFEfood_A_v_CKernel))


initialize_param() minor warning: Did nothing; already initialized parameter reference_metric::M_PI
initialize_param() minor warning: Did nothing; already initialized parameter reference_metric::xmin
initialize_param() minor warning: Did nothing; already initialized parameter reference_metric::xmax
initialize_param() minor warning: Did nothing; already initialized parameter reference_metric::ymin
initialize_param() minor warning: Did nothing; already initialized parameter reference_metric::ymax
initialize_param() minor warning: Did nothing; already initialized parameter reference_metric::zmin
initialize_param() minor warning: Did nothing; already initialized parameter reference_metric::zmax


## Step 2C: Import densitized Poynting flux initial data conversion C function

The [GiRaFFEfood_HO.GiRaFFEfood_HO.py](../edit/GiRaFFEfood_HO/GiRaFFEfood_HO.py) NRPy+ module does the following:

1. Set up Exact Wald initial data quantities in the **Cartesian basis**, as [documented here](Tutorial-GiRaFFEfood_HO_Aligned_Rotator.ipynb).
2. Convert initial magnetic fields and Valencia 3-velocities into densitized Poynting flux initial data.

We only use the second functionality here (for now).


In [3]:
# Step 2: Create the C code output kernel.
gri.glb_gridfcs_list = []
import GiRaFFEfood_HO.GiRaFFEfood_HO as gfho
gfho.GiRaFFEfood_HO_ID_converter()
# To best format this for the ETK, we'll need to register this gridfunction.
StildeD = ixp.register_gridfunctions_for_single_rank1("EVOL","StildeD")
GiRaFFE_S_to_print = [\
                      lhrh(lhs=gri.gfaccess("out_gfs","StildeD0"),rhs=gfho.StildeD[0]),\
                      lhrh(lhs=gri.gfaccess("out_gfs","StildeD1"),rhs=gfho.StildeD[1]),\
                      lhrh(lhs=gri.gfaccess("out_gfs","StildeD2"),rhs=gfho.StildeD[2]),\
                     ]

GiRaFFE_S_CKernel = fin.FD_outputC("returnstring",GiRaFFE_S_to_print,params="outCverbose=False")

# Format the code within a C loop over cctkGH
GiRaFFE_S_looped = lp.loop(["i2","i1","i0"],["0","0","0"],["cctk_lsh[2]","cctk_lsh[1]","cctk_lsh[0]"],\
                                   ["1","1","1"],["#pragma omp parallel for","",""],"",\
                                   GiRaFFE_S_CKernel.replace("time","cctk_time"))

# Step 4: Write the C code kernel to file.
with open("GiRaFFE_standalone/GiRaFFEfood_HO_Stilde.h", "w") as file:
    file.write(str(GiRaFFE_S_looped))


initialize_param() minor warning: Did nothing; already initialized parameter reference_metric::M_PI
initialize_param() minor warning: Did nothing; already initialized parameter reference_metric::xmin
initialize_param() minor warning: Did nothing; already initialized parameter reference_metric::xmax
initialize_param() minor warning: Did nothing; already initialized parameter reference_metric::ymin
initialize_param() minor warning: Did nothing; already initialized parameter reference_metric::ymax
initialize_param() minor warning: Did nothing; already initialized parameter reference_metric::zmin
initialize_param() minor warning: Did nothing; already initialized parameter reference_metric::zmax


## Step 3A: Output BSSN RHS expressions

In [4]:
gri.glb_gridfcs_list = [] # This is necessary because, since this was originally designed as two ETK thorns,
                          # some gridfunctions are registered twice.

import GiRaFFE_HO.GiRaFFE_Higher_Order_v2 as gho
gho.GiRaFFE_Higher_Order_v2()

# Set the finite differencing order to 4.
par.set_parval_from_str("finite_difference::FD_CENTDERIVS_ORDER", 2)

# Create the C code output kernel.
# Here, "Prereqs" refers to quantities that must be finite-difference to construct the RHSs.
Prereqs_to_print = [\
                   lhrh(lhs=gri.gfaccess("out_gfs","AevolParen"),rhs=gho.AevolParen),\
                   lhrh(lhs=gri.gfaccess("out_gfs","PevolParenU0"),rhs=gho.PevolParenU[0]),\
                   lhrh(lhs=gri.gfaccess("out_gfs","PevolParenU1"),rhs=gho.PevolParenU[1]),\
                   lhrh(lhs=gri.gfaccess("out_gfs","PevolParenU2"),rhs=gho.PevolParenU[2]),\
                   lhrh(lhs=gri.gfaccess("out_gfs","SevolParenUD00"),rhs=gho.SevolParenUD[0][0]),\
                   lhrh(lhs=gri.gfaccess("out_gfs","SevolParenUD01"),rhs=gho.SevolParenUD[0][1]),\
                   lhrh(lhs=gri.gfaccess("out_gfs","SevolParenUD02"),rhs=gho.SevolParenUD[0][2]),\
                   lhrh(lhs=gri.gfaccess("out_gfs","SevolParenUD10"),rhs=gho.SevolParenUD[1][0]),\
                   lhrh(lhs=gri.gfaccess("out_gfs","SevolParenUD11"),rhs=gho.SevolParenUD[1][1]),\
                   lhrh(lhs=gri.gfaccess("out_gfs","SevolParenUD12"),rhs=gho.SevolParenUD[1][2]),\
                   lhrh(lhs=gri.gfaccess("out_gfs","SevolParenUD20"),rhs=gho.SevolParenUD[2][0]),\
                   lhrh(lhs=gri.gfaccess("out_gfs","SevolParenUD21"),rhs=gho.SevolParenUD[2][1]),\
                   lhrh(lhs=gri.gfaccess("out_gfs","SevolParenUD22"),rhs=gho.SevolParenUD[2][2]),\
                   ]

metric_quantities_to_print = [\
                              lhrh(lhs=gri.gfaccess("out_gfs","gammaUU00"),rhs=gho.gammaUU[0][0]),\
                              lhrh(lhs=gri.gfaccess("out_gfs","gammaUU01"),rhs=gho.gammaUU[0][1]),\
                              lhrh(lhs=gri.gfaccess("out_gfs","gammaUU02"),rhs=gho.gammaUU[0][2]),\
                              lhrh(lhs=gri.gfaccess("out_gfs","gammaUU11"),rhs=gho.gammaUU[1][1]),\
                              lhrh(lhs=gri.gfaccess("out_gfs","gammaUU12"),rhs=gho.gammaUU[1][2]),\
                              lhrh(lhs=gri.gfaccess("out_gfs","gammaUU22"),rhs=gho.gammaUU[2][2]),\
                              lhrh(lhs=gri.gfaccess("out_gfs","gammadet"),rhs=gho.gammadet),\
                             ]

# To best format this for the ETK, we'll need to register these gridfunctions.
Stilde_rhsD = ixp.register_gridfunctions_for_single_rank1("AUX","Stilde_rhsD")
A_rhsD = ixp.register_gridfunctions_for_single_rank1("AUX","A_rhsD")
psi6Phi_rhs = gri.register_gridfunctions("AUX","psi6Phi_rhs")
Conservs_to_print = [\
                     lhrh(lhs=gri.gfaccess("out_gfs","Stilde_rhsD0"),rhs=gho.Stilde_rhsD[0]),\
                     lhrh(lhs=gri.gfaccess("out_gfs","Stilde_rhsD1"),rhs=gho.Stilde_rhsD[1]),\
                     lhrh(lhs=gri.gfaccess("out_gfs","Stilde_rhsD2"),rhs=gho.Stilde_rhsD[2]),\
                     lhrh(lhs=gri.gfaccess("out_gfs","A_rhsD0"),rhs=gho.A_rhsD[0]),\
                     lhrh(lhs=gri.gfaccess("out_gfs","A_rhsD1"),rhs=gho.A_rhsD[1]),\
                     lhrh(lhs=gri.gfaccess("out_gfs","A_rhsD2"),rhs=gho.A_rhsD[2]),\
                     lhrh(lhs=gri.gfaccess("out_gfs","psi6Phi_rhs"),rhs=gho.psi6Phi_rhs),\
                    ]

Prereqs_CKernel = fin.FD_outputC("returnstring",Prereqs_to_print,params="outCverbose=False")
#Prereqs_CKernel = "const double u0 = u0GF[CCTK_GFINDEX3D(cctkGH, i0,i1,i2)];\n" + Prereqs_CKernel
metric_quantities_CKernel = fin.FD_outputC("returnstring",metric_quantities_to_print,params="outCverbose=False")
Conservs_CKernel = fin.FD_outputC("returnstring",Conservs_to_print,params="outCverbose=False")
#Conservs_CKernel = "const double u0 = u0GF[CCTK_GFINDEX3D(cctkGH, i0,i1,i2)];\n" + Conservs_CKernel

Prereqs_looped = lp.loop(["i2","i1","i0"],["0","0","0"],\
                           ["cctk_lsh[2]","cctk_lsh[1]","cctk_lsh[0]"],\
                           ["1","1","1"],["#pragma omp parallel for","",""],"",\
                           Prereqs_CKernel.replace("time","cctk_time"))

metric_quantities_looped = lp.loop(["i2","i1","i0"],["0","0","0"],\
                                     ["cctk_lsh[2]","cctk_lsh[1]","cctk_lsh[0]"],\
                                     ["1","1","1"],["#pragma omp parallel for","",""],"",\
                                     metric_quantities_CKernel.replace("time","cctk_time"))

Conservs_looped = lp.loop(["i2","i1","i0"],["cctk_nghostzones[2]","cctk_nghostzones[1]","cctk_nghostzones[0]"],\
                            ["cctk_lsh[2]-cctk_nghostzones[2]","cctk_lsh[1]-cctk_nghostzones[1]",\
                             "cctk_lsh[0]-cctk_nghostzones[0]"],\
                            ["1","1","1"],["#pragma omp parallel for","",""],"",\
                            Conservs_CKernel.replace("time","cctk_time"))


## Steps 3B & 2B: Output A-to-B expressions

These expressions are used to calculate the magnetic fields from the vector potential. See [here](Tutorial-ETK_thorn-GiRaFFE_Higher_Order_v2.ipynb#step1p6) for more information on why they are implemented the way they are. 

In [ ]:
# The A-to-B driver

# Import the Levi-Civita symbol and build the corresponding tensor.
# We already have a handy function to define the Levi-Civita symbol in WeylScalars
import WeylScal4NRPy.WeylScalars_Cartesian as weyl
LeviCivitaDDD = weyl.define_LeviCivitaSymbol_rank3()
LeviCivitaUUU = ixp.zerorank3()
for i in range(DIM):
    for j in range(DIM):
        for k in range(DIM):
            LCijk = LeviCivitaDDD[i][j][k]
            #LeviCivitaDDD[i][j][k] = LCijk * sp.sqrt(gho.gammadet)
            LeviCivitaUUU[i][j][k] = LCijk / sp.sqrt(gho.gammadet)

AD_dD = ixp.declarerank2("AD_dD","nosym")
BU = ixp.zerorank1() # BU is already registered as a gridfunction, but we need to zero its values and declare it in this scope.
# We can use this function to compactly reset to expressions to print at each FD order.
def set_BU_to_print():
    return [lhrh(lhs=gri.gfaccess("out_gfs","BU0"),rhs=BU[0]),\
            lhrh(lhs=gri.gfaccess("out_gfs","BU1"),rhs=BU[1]),\
            lhrh(lhs=gri.gfaccess("out_gfs","BU2"),rhs=BU[2])]            

for i in range(DIM):
    for j in range(DIM):
        for k in range(DIM):
            BU[i] += LeviCivitaUUU[i][j][k] * AD_dD[k][j]

# We'll lower the FD order at each stage and write to a new file.
par.set_parval_from_str("finite_difference::FD_CENTDERIVS_ORDER", 10)
fin.FD_outputC("GiRaFFE_standalone/B_from_A_10.h",set_BU_to_print(),params="outCverbose=False")

par.set_parval_from_str("finite_difference::FD_CENTDERIVS_ORDER", 8)
fin.FD_outputC("GiRaFFE_standalone/B_from_A_8.h",set_BU_to_print(),params="outCverbose=False")

par.set_parval_from_str("finite_difference::FD_CENTDERIVS_ORDER", 6)
fin.FD_outputC("GiRaFFE_standalone/B_from_A_6.h",set_BU_to_print(),params="outCverbose=False")

par.set_parval_from_str("finite_difference::FD_CENTDERIVS_ORDER", 4)
fin.FD_outputC("GiRaFFE_standalone/B_from_A_4.h",set_BU_to_print(),params="outCverbose=False")

par.set_parval_from_str("finite_difference::FD_CENTDERIVS_ORDER", 2)
fin.FD_outputC("GiRaFFE_standalone/B_from_A_2.h",set_BU_to_print(),params="outCverbose=False")

# For the outermost points, we'll need a separate file for each face. 
# These will correspond to an upwinded and a downwinded file for each direction.
AD_ddnD = ixp.declarerank2("AD_ddnD","nosym")
for i in range(DIM):
    BU[i] = 0
    for j in range(DIM):
        for k in range(DIM):
            if j is 0:
                BU[i] += LeviCivitaUUU[i][j][k] * AD_ddnD[k][j]
            else:
                BU[i] += LeviCivitaUUU[i][j][k] * AD_dD[k][j]

fin.FD_outputC("GiRaFFE_standalone/B_from_A_2x0D.h",set_BU_to_print(),params="outCverbose=False")

AD_dupD = ixp.declarerank2("AD_dupD","nosym")
for i in range(DIM):
    BU[i] = 0
    for j in range(DIM):
        for k in range(DIM):
            if j is 0:
                BU[i] += LeviCivitaUUU[i][j][k] * AD_dupD[k][j]
            else:
                BU[i] += LeviCivitaUUU[i][j][k] * AD_dD[k][j]

fin.FD_outputC("GiRaFFE_standalone/B_from_A_2x0U.h",set_BU_to_print(),params="outCverbose=False")

for i in range(DIM):
    BU[i] = 0
    for j in range(DIM):
        for k in range(DIM):
            if j is 1:
                BU[i] += LeviCivitaUUU[i][j][k] * AD_ddnD[k][j]
            else:
                BU[i] += LeviCivitaUUU[i][j][k] * AD_dD[k][j]

fin.FD_outputC("GiRaFFE_standalone/B_from_A_2x1D.h",set_BU_to_print(),params="outCverbose=False")
for i in range(DIM):
    BU[i] = 0
    for j in range(DIM):
        for k in range(DIM):
            if j is 1:
                BU[i] += LeviCivitaUUU[i][j][k] * AD_dupD[k][j]
            else:
                BU[i] += LeviCivitaUUU[i][j][k] * AD_dD[k][j]

fin.FD_outputC("GiRaFFE_standalone/B_from_A_2x1U.h",set_BU_to_print(),params="outCverbose=False")

for i in range(DIM):
    BU[i] = 0
    for j in range(DIM):
        for k in range(DIM):
            if j is 2:
                BU[i] += LeviCivitaUUU[i][j][k] * AD_ddnD[k][j]
            else:
                BU[i] += LeviCivitaUUU[i][j][k] * AD_dD[k][j]

fin.FD_outputC("GiRaFFE_standalone/B_from_A_2x2D.h",set_BU_to_print(),params="outCverbose=False")
for i in range(DIM):
    BU[i] = 0
    for j in range(DIM):
        for k in range(DIM):
            if j is 2:
                BU[i] += LeviCivitaUUU[i][j][k] * AD_dupD[k][j]
            else:
                BU[i] += LeviCivitaUUU[i][j][k] * AD_dD[k][j]

fin.FD_outputC("GiRaFFE_standalone/B_from_A_2x2U.h",set_BU_to_print(),params="outCverbose=False")


## Step 4: Apply singular, curvilinear coordinate boundary conditions [as documented in the corresponding NRPy+ tutorial module](Tutorial-Start_to_Finish-Curvilinear_BCs.ipynb)

In [ ]:
# Declaring StildeD as a gridfunction is unnecessary in GiRaFFe_HO. While it was declared in GiRaFFEfood_HO,
# those have since been cleared to avoid conflict; so, we re-declare it here.
StildeD = ixp.register_gridfunctions_for_single_rank1("EVOL","StildeD")
import CurviBoundaryConditions.CurviBoundaryConditions as cbcs
cbcs.Set_up_CurviBoundaryConditions()

# BrillLindquist_Playground.c: The Main C Code

In [ ]:
%%writefile GiRaFFE_standalone/GiRaFFE_standalone.c
// Step P1: Import needed header files
#include "NGHOSTS.h" // A NRPy+-generated file, which is set based on FD_CENTDERIVS_ORDER.
#include "stdio.h"
#include "stdlib.h"
#include "math.h"
#include "time.h"

// Step P2: Add needed #define's to set data type, the IDX4() macro, and the gridfunctions
// Step P2a: set REAL=double, so that all floating point numbers are stored to at least ~16 significant digits.
#define REAL double

// Step P3: Set free parameters
// Step P3a: Free parameters for the numerical grid
// Cartesian coordinates parameters
const REAL xmin = -10.,xmax=10.;
const REAL ymin = -10.,ymax=10.;
const REAL zmin = -10.,zmax=10.;

// Time coordinate parameters
const REAL t_final =  7.5; /* Final time is set so that at t=t_final, 
                            * data at the origin have not been corrupted 
                            * by the approximate outer boundary condition */
REAL CFL_FACTOR = 0.5; // Set the CFL Factor

// Step P3b: Free parameters for the spacetime evolution
// Step P5: Set free parameters for the (Brill-Lindquist) initial data

// Step P6: Declare the IDX4(gf,i,j,k) macro, which enables us to store 4-dimensions of
//          data in a 1D array. In this case, consecutive values of "i" 
//          (all other indices held to a fixed value) are consecutive in memory, where 
//          consecutive values of "j" (fixing all other indices) are separated by 
//          Nxx_plus_2NGHOSTS[0] elements in memory. Similarly, consecutive values of
//          "k" are separated by Nxx_plus_2NGHOSTS[0]*Nxx_plus_2NGHOSTS[1] in memory, etc.
#define IDX4(g,i,j,k) \
( (i) + Nxx_plus_2NGHOSTS[0] * ( (j) + Nxx_plus_2NGHOSTS[1] * ( (k) + Nxx_plus_2NGHOSTS[2] * (g) ) ) )
#define IDX3(i,j,k) ( (i) + Nxx_plus_2NGHOSTS[0] * ( (j) + Nxx_plus_2NGHOSTS[1] * (k) ) )
// Assuming idx = IDX3(i,j,k). Much faster if idx can be reused over and over:
#define IDX4pt(g,idx)   ( (idx) + (Nxx_plus_2NGHOSTS[0]*Nxx_plus_2NGHOSTS[1]*Nxx_plus_2NGHOSTS[2]) * (g) )
